<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be sometime that can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [66]:
# Tensorflow imports
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential # This is the functional API

In [67]:
import numpy as np

from skimage.io import imread_collection
from skimage.transform import resize

from sklearn.model_selection import train_test_split

import pathlib

resnet = ResNet50(weights='imagenet', include_top=False)

In [68]:
# load in images
mountain_raw = './module2-convolutional-neural-networks/data/mountain/*.jpg'
forest_raw = './module2-convolutional-neural-networks/data/forest/*.jpg'
mountain = imread_collection(mountain_raw)
forest = imread_collection(forest_raw)

# look at the data
print(len(mountain), len(forest), mountain[0].shape, forest[0].shape)

# concatenate the data

# mountains/forest
mountain = mountain.concatenate()
forest = forest.concatenate()

# x concat
x = np.concatenate([forest, mountain])

# y concat and zeros
y = np.concatenate([np.zeros(forest.shape[0]), np.ones(mountain.shape[0])])

print(x.shape, y.shape)

374 328 (256, 256, 3) (256, 256, 3)
(702, 256, 256, 3) (702,)


In [69]:
# Train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(526, 256, 256, 3) (176, 256, 256, 3) (526,) (176,)


In [70]:
ResNet50 = ResNet50(input_shape=(256, 256, 3), weights='imagenet', include_top=False)
print('Done')

Done


In [71]:
# Remove connected layers
for layer in ResNet50.layers:
    layer.trainable = False

In [72]:
x = ResNet50.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

## Instatiate Model

In [73]:
model = Model(ResNet50.input, predictions)
model.summary()

[0][0]        
__________________________________________________________________________________________________
conv4_block1_2_bn (BatchNormali (None, 16, 16, 256)  1024        conv4_block1_2_conv[0][0]        
__________________________________________________________________________________________________
conv4_block1_2_relu (Activation (None, 16, 16, 256)  0           conv4_block1_2_bn[0][0]          
__________________________________________________________________________________________________
conv4_block1_0_conv (Conv2D)    (None, 16, 16, 1024) 525312      conv3_block4_out[0][0]           
__________________________________________________________________________________________________
conv4_block1_3_conv (Conv2D)    (None, 16, 16, 1024) 263168      conv4_block1_2_relu[0][0]        
__________________________________________________________________________________________________
conv4_block1_0_bn (BatchNormali (None, 16, 16, 1024) 4096        conv4_block1_0_conv[0][0]    

## Fit Model

In [74]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [76]:
model.fit(x_train, y_train, epochs=5, validation_split=.25)

Train on 394 samples, validate on 132 samples
Epoch 1/5
394/394 [==============================] - 73s 184ms/sample - loss: 0.4794 - accuracy: 0.7919 - val_loss: 0.0847 - val_accuracy: 0.9621
Epoch 2/5
394/394 [==============================] - 67s 171ms/sample - loss: 0.1524 - accuracy: 0.9442 - val_loss: 0.0963 - val_accuracy: 0.9545
Epoch 3/5
394/394 [==============================] - 67s 171ms/sample - loss: 0.0390 - accuracy: 0.9898 - val_loss: 0.0229 - val_accuracy: 0.9924
Epoch 4/5
394/394 [==============================] - 66s 166ms/sample - loss: 0.0581 - accuracy: 0.9746 - val_loss: 0.0377 - val_accuracy: 0.9848
Epoch 5/5
394/394 [==============================] - 64s 163ms/sample - loss: 0.0119 - accuracy: 0.9975 - val_loss: 0.0222 - val_accuracy: 0.9924


In [77]:
model.evaluate(x_test, y_test, verbose=2)

176/1 - 11s - loss: 0.0103 - accuracy: 1.0000


[0.010808897797356953, 1.0]

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language